In [1]:
import pandas as pd
import csv

import os, os.path
import sys

# to move files from one directory to another
import shutil

In [2]:
perry_dir = "/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/"
plots_dir = "/Users/hn/Documents/01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/"

In [3]:
# choices_set_1 = pd.read_excel(io=param_dir + "set1_PerryandCo.xlsx", sheet_name=0)
# response_set_1 = pd.read_excel(io=param_dir + "Perry_and_Co_Responses_set_1.xlsx", sheet_name=0)

In [4]:
choices_set_1_xl = pd.ExcelFile(perry_dir + "set1_PerryandCo.xlsx")
choises_set_1_sheet_names = choices_set_1_xl.sheet_names  # see all sheet names

response_set_1_xl = pd.ExcelFile(perry_dir + "Perry_and_Co_Responses_set_1.xlsx")
response_set_1_sheet_names = response_set_1_xl.sheet_names  # see all sheet names


In [5]:
print (choises_set_1_sheet_names)
print (response_set_1_sheet_names)

['extended_1', 'extended_2', 'extended_3', 'extended_4', 'extended_5', 'extended_6']
['S1 - F6', 'S1 - F5', 'S1 - F4', 'S1 - F3', 'S1 - F2', 'S1 - F1']


In [6]:
a_choice_sheet = choices_set_1_xl.parse(choises_set_1_sheet_names[5])
sample_response = response_set_1_xl.parse(response_set_1_sheet_names[5])

In [7]:
print (list(a_choice_sheet.columns))

['Question_in_set', 'Question_overall', 'ID', 'NDVI_TS_Name', 'corrected_RGB', 'TOA_RGB', 'latitude', 'longitude', 'CropTyp', 'Irrigtn', 'DataSrc', 'Acres', 'ExctAcr', 'LstSrvD', 'county']


## Count number of questions

In [8]:
question_count = 0

for a_choice_sheet in choises_set_1_sheet_names:
    
    # read a damn sheet
    a_choice_sheet = choices_set_1_xl.parse(a_choice_sheet)

    # add them to the damn list
    question_count += a_choice_sheet.shape[0]

print('There are [{ques_count}] questions.'.format(ques_count=question_count))

There are [294] questions.


In [9]:
# unwanted_opinions = ["kirtir@wsu.edu"]

In [10]:
sample_response.head(1)

,Timestamp,Email Address,"QUESTION 1 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/45.99256829,-119.34481065/@45.99256829,-119.34481065,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?,Comments,"QUESTION 2 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.07535056,-119.65017789/@46.07535056,-119.65017789,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.1,Comments.1,"QUESTION 3 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/45.96712978,-119.37401358/@45.96712978,-119.37401358,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.2,...,Comments.46,"QUESTION 48 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/47.24441649,-119.80872437/@47.24441649,-119.80872437,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.47,Comments.47,"QUESTION 49 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.29637591,-118.8213993/@46.29637591,-118.8213993,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.48,Comments.48,"QUESTION 50 : How would you classify? \n Ignore any late-fall/winter signatures such as fall planting of a crop, winter cover crops, or volunteers/weeds. \n http://www.google.com/maps/place/46.39197279,-119.23327008/@46.39197279,-119.23327008,16z/data=!3m1!1e3 [Choose one option: ]",Discuss as a full group?.49,Comments.49
0,2022-02-02 06:36:57.993,kirtir@wsu.edu,Single Crop,Check for Yes,Weeds or volunteers with herbicide given timi...,Single Crop,NaN,NaN,Single Crop,Check for Yes,...,NaN,Unsure,Check for Yes,NaN,Unsure,Check for Yes,NaN,Single Crop,NaN,NaN


### Discussion Table

In [11]:
## Define the damn output dataframe

discuss_table = pd.DataFrame(columns=['Discuss_Form', 'Discuss_Question', 'ID', "Perry", "Andrew", "Tim", "Kirti"], 
                         index=range(question_count))
discuss_table.head(1)
curr_row = 0

extended_choices_discuss = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choise_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choise_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    no_people_in_sheet = a_response_sheet.shape[0]
    
    extended_choices_discuss = pd.concat([extended_choices_discuss, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if ("Discuss" in a_col_name) and (a_response_sheet[a_col_name].isna().sum()<no_people_in_sheet):
            if a_col_name.split(".")[-1].isdigit():
                question_number = str(int(a_col_name.split(".")[-1])+1)
            else:
                question_number = "1"
            
            discuss_table.loc[curr_row, "Discuss_Form"] = sheet_numeric_part
            discuss_table.loc[curr_row, "Discuss_Question"] = question_number
#            discuss_table.loc[curr_row, "opinion_count"] = len(a_response_sheet[a_col_name].unique())
            discuss_table.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
            discuss_table.loc[curr_row, "Perry"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            discuss_table.loc[curr_row, "Andrew"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            discuss_table.loc[curr_row, "Tim"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            if "kirtir@wsu.edu" in list(a_response_sheet["Email Address"]):
                discuss_table.loc[curr_row, "Kirti"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "kirtir@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1
            
# drop rows where all row is NaNs
discuss_table = discuss_table.dropna(axis=0, how='all')

### Replace NaN with "No"
discuss_table = discuss_table.fillna("No")
discuss_table_extended = pd.merge(discuss_table, extended_choices_discuss, on=['ID'], how='left')
discuss_table_extended.shape

(32, 21)

### Comments

In [12]:
## Define the damn output dataframe

comment_table = pd.DataFrame(columns=['Comment_Form', 'Comment_Question', 'ID', 
                                      "PerryComment", "AndrewComment", "TimComment", "KirtiComment"], 
                         index=range(question_count))
comment_table.head(1)
curr_row = 0

extended_choices_comment = pd.DataFrame()

###### populate the output datafrme

for response_sheet_name in response_set_1_sheet_names:
    
    # pick up the numeric part of the sheet names from google forms sheets
    # this is the form number as well.
    sheet_numeric_part = [s for s in response_sheet_name.split()[-1] if s.isdigit()]
    sheet_numeric_part = "".join(sheet_numeric_part)
    
    # form sheet names of choices excel sheets
    choise_sheet_name = "extended_" + sheet_numeric_part
    
    a_choice_sheet = choices_set_1_xl.parse(choise_sheet_name)
    a_response_sheet = response_set_1_xl.parse(response_sheet_name)
    
    no_people_in_sheet = a_response_sheet.shape[0]
    
    extended_choices_comment = pd.concat([extended_choices_comment, a_choice_sheet])
    #
    # drop unwanted opinions, i.e. keep experts opinions
    #
    # a_response_sheet = a_response_sheet[~(a_response_sheet["Email Address"].isin(unwanted_opinions))].copy()
    
    for a_col_name in a_response_sheet.columns:
        if ("Comment" in a_col_name) and (a_response_sheet[a_col_name].isna().sum()<no_people_in_sheet):
            if a_col_name.split(".")[-1].isdigit():
                question_number = str(int(a_col_name.split(".")[-1])+1)
            else:
                question_number = "1"
            
            comment_table.loc[curr_row, "Comment_Form"] = sheet_numeric_part
            comment_table.loc[curr_row, "Comment_Question"] = question_number

            
            comment_table.loc[curr_row, "ID"] = a_choice_sheet.loc[int(question_number)-1, "ID"]
            
            comment_table.loc[curr_row, "PerryComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "pbeale@agr.wa.gov"][a_col_name].values[0]
            
            comment_table.loc[curr_row, "AndrewComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "andrew.mcguire@wsu.edu"][a_col_name].values[0]
            
            comment_table.loc[curr_row, "TimComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "twaters@wsu.edu"][a_col_name].values[0]
            
            if "kirtir@wsu.edu" in list(a_response_sheet["Email Address"]):
                comment_table.loc[curr_row, "KirtiComment"] = a_response_sheet[a_response_sheet["Email Address"] == \
                                                                 "kirtir@wsu.edu"][a_col_name].values[0]
            
            curr_row += 1
            
# drop rows where all row is NaNs
comment_table = comment_table.dropna(axis=0, how='all')

### Replace NaN with "No"
comment_table = comment_table.fillna("No")

comment_table_extended = pd.merge(comment_table, extended_choices_comment, on=['ID'], how='left')
comment_table_extended.shape

(34, 21)

In [13]:
output_together = pd.merge(discuss_table_extended, comment_table_extended, on=['ID'], how='outer')
output_together = output_together.fillna("No")
output_together.shape

(50, 41)

In [14]:
output_together.tail(2)

,Discuss_Form,Discuss_Question,ID,Perry,Andrew,Tim,Kirti,Question_in_set_x,Question_overall_x,NDVI_TS_Name_x,...,TOA_RGB_y,latitude_y,longitude_y,CropTyp_y,Irrigtn_y,DataSrc_y,Acres_y,ExctAcr_y,LstSrvD_y,county_y
48,No,No,108842_WSDA_SF_2017,No,No,No,No,No,No,No,...,108842_WSDA_SF_2017_47.090746_-119.81387885_TO...,47.090746,-119.813879,wheat,center pivot,wsda,32.0,32.301584,2017/10/16,Grant
49,No,No,109375_WSDA_SF_2017,No,No,No,No,No,No,No,...,109375_WSDA_SF_2017_47.13175543_-119.62366484_...,47.131755,-119.623665,wheat fallow,center pivot,wsda,49.0,48.55717,2017/08/28,Grant


In [15]:
print (comment_table.ID.isin(discuss_table.ID).sum())
print (discuss_table.ID.isin(comment_table.ID).sum())

16
16


## Fix the Form and Question numbers

In [16]:
for row in range(output_together.shape[0]):
    if output_together.loc[row, "Discuss_Form"]=="No":
        output_together.loc[row, "Discuss_Form"] = output_together.loc[row, "Comment_Form"]
        output_together.loc[row, "Discuss_Question"] = output_together.loc[row, "Comment_Question"]
        output_together.loc[row, "NDVI_TS_Name_x"] = output_together.loc[row, "NDVI_TS_Name_y"]
        output_together.loc[row, "CropTyp_x"] = output_together.loc[row, "CropTyp_y"]

### Create a long PDF of fields where discussion is requested

In [17]:
import subprocess
os.getcwd()

'/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/Perry_and_Co'

In [18]:
output_together.sort_values(by=['CropTyp_x'],
                           ascending = [True],
                           inplace=True)
output_together.reset_index(inplace=True, drop=True)


os.chdir('/Users/hn/Documents/01_research_data/NASA/Perry_and_Co/set_1_PDFs/')

with open("set_1_Discussion_and_Comments.tex","w") as file:
    file.write("\\documentclass{article}\n")

    file.write("\\usepackage[osf,sc]{mathpazo}\n")
    file.write("\\usepackage{color}\n")

    file.write("\\usepackage{mathtools,hyperref}\n")
    file.write("\\hypersetup{\n")
    file.write("      colorlinks=true,\n")
    file.write("      linkcolor=cyan,\n")
    file.write("      filecolor=cyan,\n")
    file.write("      urlcolor=mgreen,\n")
    file.write("      citecolor=cyan}\n")

    file.write("\\usepackage[margin=1in]{geometry}\n")
    file.write("\\usepackage[T1]{fontenc}\n")
    
    file.write("\\usepackage{caption}\n")
    file.write("\\captionsetup{justification= raggedright, singlelinecheck = false}\n")

    file.write("\\title{\\bf Set 1 - Discussion and Comments}\n")
    file.write("\\author{}\n")
    file.write("\\date{}\n")


    file.write("\\begin{document}\n")
    file.write("\\maketitle\n")

    file.write("\\section{Discussion and Comments}\n")
    
    file.write("\n")
    file.write("\n")
    
    for a_row in range(output_together.shape[0]):
        curr_row = output_together.loc[a_row]
        TS_file_name = curr_row["NDVI_TS_Name_x"]
        file.write("\\begin{figure*}[ht]\n")
        file.write("\\centering\n")
        file.write("\\includegraphics[width=1\\textwidth]{/Users/hn/Documents/" + \
                     "01_research_data/NASA/snapshots/TS/06_snapshot_flat_PNG/" + \
                         TS_file_name + "}\n")
        file.write("\\caption[]{" + \
                   "\\textbf{\\color{red}{" + curr_row["CropTyp_x"] + "}}" + \
                   ", ID: " + curr_row["ID"].replace("_", "\_") + \
                   ", Form: " + curr_row["Discuss_Form"] + \
                   ", Question: " + curr_row["Discuss_Question"] + "." + \
                   "}\n")
        file.write("\\label{fig:figure" + str(a_row) + "}\n")
        file.write("\\end{figure*}\n")
        file.write("\n")
        
        
        file.write("\\noindent \\textbf{Discuss?}" + "\\\\\n")
        file.write("\\noindent Perry: ~~~~" + str(curr_row["Perry"]) + "\\\\\n")
        file.write("Andrew: " + str(curr_row["Andrew"]) + "\\\\\n")
        file.write("Tim :~~~~~~~" + str(curr_row["Tim"]) +  "\\\\\n")
        file.write("Kirti: ~~~~~~" + str(curr_row["Kirti"]) + "\\\\\n")
        
        file.write("\n")
        file.write("\n")
        file.write("\\noindent \\textbf{Comments}" + "\\\\\n")
        file.write("\\noindent Perry: ~~~~" + str(curr_row["PerryComment"]) + "\\\\\n")
        file.write("Andrew: " + str(curr_row["AndrewComment"]) + "\\\\\n")
        file.write("Tim: ~~~~~~~" + str(curr_row["TimComment"]) +  "\\\\\n")
        file.write("Kirti:  ~~~~~~" + str(curr_row["KirtiComment"]) + "\\\\\n")
                
        file.write("\\clearpage\n")
        
        file.write("\n")
        file.write("\n")
        file.write("\n")
        

    file.write("\\end{document}\n")